In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
gis = arcgis.GIS()

from IPython.display import display, Markdown

In [2]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_data     = os.path.join(dir_working, "data"        )
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "inputs"      )

ClassParameters    = os.path.join(dir_inputs, r"class_parameters.csv"     )

ProcessGDB = "process.gdb"

#name of new data features
AnalysisAreas      = os.path.join(dir_process, "analysisareas.shp"        )
AnalysisAreas_IDs  = os.path.join(dir_process, "analysisareas_ids.shp"    )
ParcelsAA          = os.path.join(dir_process, "parcelsaa.shp"            )

AnalysisAreasTable = os.path.join(dir_results, "areas.csv"                )

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)


E:\GitHub\Centers-Capacity-Tool


In [3]:
AnalysisAreas

'E:\\GitHub\\Centers-Capacity-Tool\\intermediate\\analysisareas.shp'

In [4]:
sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
sdf_AA

,FID,AreaName,AreaType,Area,ClassName,Shape_Leng,Shape_Area,AnalysisAr,Analysis_1,SHAPE
0,0,PROVO CENTRAL CRT NC,,Utah County,CRT|NA,3312.944042,249304.722110,PROVO CENTRAL CRT NC,CRT|NA,"{""rings"": [[[443768.9589999998, 4452453.058], ..."
1,1,OREM CENTRAL CRT NC,,Utah County,CRT|NA,3369.672280,903577.621946,OREM CENTRAL CRT NC,CRT|NA,"{""rings"": [[[438326.58999999985, 4459639.39000..."
2,2,AMERICAN FORK CRT NC,,Utah County,CRT|NA,2570.761620,278930.897498,AMERICAN FORK CRT NC,CRT|NA,"{""rings"": [[[430849.71740000043, 4469533.5944]..."
3,3,LEHI CRT NC,,Utah County,CRT|NA,2134.667772,259008.671655,LEHI CRT NC,CRT|NA,"{""rings"": [[[423912.7335000001, 4474824.7914],..."
4,4,DRAPER CRT NC,,WFRC MPO,CRT|NA,4480.383277,310214.302238,DRAPER CRT NC,CRT|NA,"{""rings"": [[[423670.5016999999, 4485868.4681],..."
...,...,...,...,...,...,...,...,...,...,...
206,206,Brigham City Main Street,City Center,WFRC MPO,NONTOD|City Center,1985.655585,239851.155571,Brigham City Main Street,NONTOD|City Center,"{""rings"": [[[415528.0384999998, 4596136.477], ..."
207,207,Midland Drive - 3500 West,City Center,WFRC MPO,NONTOD|City Center,243.559404,0.256620,Midland Drive - 3500 West,NONTOD|City Center,"{""rings"": [[[410819.7604, 4557518.294199999], ..."
208,208,West Roy Town Center,City Center,WFRC MPO,NONTOD|City Center,243.559404,0.256620,West Roy Town Center,NONTOD|City Center,"{""rings"": [[[410819.7604, 4557518.294199999], ..."
209,209,Syracuse 2000 West - North,City Center,WFRC MPO,NONTOD|City Center,383.465228,9603.551349,Syracuse 2000 West - North,NONTOD|City Center,"{""rings"": [[[410459.86950000003, 4551736.14409..."


In [24]:
#create map centered on Salt Lake
map_areas = gis.map('Salt Lake')

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[128,128,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#classify land use
arcade_expression_aa = ("var v = $feature.Analysis_1;"
                        "if      (v=='NONTOD|Metropolitan Center') { return 'mc'    ; }"
                        "else if (v=='CRT|Metropolitan Center'   ) { return 'mc_tod'; }"
                        "else if (v=='LRT|Metropolitan Center'   ) { return 'mc_tod'; }"
                        "else if (v=='NONTOD|Urban Center'       ) { return 'uc'    ; }"
                        "else if (v=='CRT|Urban Center'          ) { return 'uc_tod'; }"
                        "else if (v=='LRT|Urban Center'          ) { return 'uc_tod'; }"
                        "else if (v=='NONTOD|City Center'        ) { return 'cc'    ; }"
                        "else if (v=='CRT|City Center'           ) { return 'cc_tod'; }"
                        "else if (v=='LRT|City Center'           ) { return 'cc_tod'; }"
                        "else if (v=='CRT|NA'                    ) { return 'nc_tod'; }"
                        "else if (v=='LRT|NA'                    ) { return 'nc_tod'; }"
                        "else                                      { return 'other' ; }")

#symbology for enrollment classes
uv_aa = [  
          {"value":"mc"    , "label":"Metropolitan Center"         , "symbol":{"type":"esriSFS","color":[128,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"mc_tod", "label":"Metropolitan Center with TOD", "symbol":{"type":"esriSFS","color":[128,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"uc"    , "label":"Urban Center"                , "symbol":{"type":"esriSFS","color":[  0,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"uc_tod", "label":"Urban Center with TOD"       , "symbol":{"type":"esriSFS","color":[  0,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"cc"    , "label":"City Center"                 , "symbol":{"type":"esriSFS","color":[255,  0,  0,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"cc_tod", "label":"City Center with TOD"        , "symbol":{"type":"esriSFS","color":[255,  0,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"nc_tod", "label":"TOD Only"                    , "symbol":{"type":"esriSFS","color":[  0,255,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"other" , "label":"Other"                       , "symbol":{"type":"esriSFS","color":[128,128,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}
        ]

#define sdf layer
sdf_AA.spatial.plot(map_widget = map_areas,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_areas.layout.height='500px'
map_areas.legend=True

#map title
display(Markdown('<h2><center>Analysis Areas</center></h2>'))

#display map
map_areas

<h2><center>Analysis Areas</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Parcels

In [6]:
sdf_ParcelsAA = pd.DataFrame.spatial.from_featureclass(ParcelsAA)
sdf_ParcelsAA.groupby(['AreaID','ClassID'], as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "Acres":[np.sum]})

,AreaID,ClassID,parcel_id,job_spaces,resunits,Acres
,,,size,sum,sum,sum
0,0,8,134.0,1366.601427,36.847993,78.918319
1,1,6,57.0,697.950149,165.891281,56.508939
2,2,8,743.0,3426.215028,537.232081,267.926221
3,3,6,331.0,0.000000,293.293031,98.052428
4,4,6,45.0,85.979645,0.000000,124.164261
...,...,...,...,...,...,...
181,155,8,819.0,3095.949425,825.692231,447.421113
182,156,6,27.0,0.000000,12.499403,2.738657
183,157,4,247.0,1309.964070,101.212094,67.876261


In [7]:
sdf_ParcelsAA['county_id']

0         4.0
1         4.0
2         4.0
3         4.0
4         4.0
         ... 
125856    2.0
125857    2.0
125858    2.0
125859    2.0
125860    2.0
Name: county_id, Length: 125861, dtype: float64

# Land Use

In [8]:
display(sdf_ParcelsAA.columns)
sdf_ParcelsAA = sdf_ParcelsAA.rename(columns={"luType": "luTypeEx"})
display(sdf_ParcelsAA.columns)

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height', 'luType',
       'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt', 'resunits',
       'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re', 'sfvalue_co',
       'value_res', 'value_com', 'ParcelAcre', 'PieceAcres', 'PiecePorti',
       'Acres', 'SHAPE'],
      dtype='object')

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE'],
      dtype='object')

In [9]:
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & (sdf_ParcelsAA['bldgtype'] == 'Residential'), 'luTypeEx'] = "Residential"
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & (sdf_ParcelsAA['bldgtype'] == 'Commercial' ), 'luTypeEx'] = "Commercial"
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & ((sdf_ParcelsAA['bldgtype'] != 'Residential') & (sdf_ParcelsAA['bldgtype'] != 'Commercial')), 'luTypeEx'] = "Developable"

In [10]:
#create map centered on Salt Lake
map_parcels_undevelopable = gis.map('Salt Lake')

countyfilter = 2


#Filter by County
sdf_ParcelsAA_lu = sdf_ParcelsAA[sdf_ParcelsAA['county_id']==countyfilter]
#sdf_ParcelsAA_lu

#classify land use
arcade_expression_lu = ("var v = $feature.luTypeEx;"
                        "if      (v=='Residential'  ) { return 'res'  ; }"
                        "else if (v=='Commercial'   ) { return 'com'  ; }"
                        "else if (v=='Developable'  ) { return 'dev'  ; }"
                        "else if (v=='Undevelopable') { return 'undev'; }"
                        "else                           { return 'other'  ; }")

#symbology for enrollment classes
uv_lu = [{"value":"res"  , "label":"Residential"  , "symbol":{"type":"esriSFS","color":[255,255,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
         {"value":"com"  , "label":"Commercial"   , "symbol":{"type":"esriSFS","color":[255,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
         {"value":"dev"  , "label":"No Building"  , "symbol":{"type":"esriSFS","color":[  0,  0,255,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
         {"value":"undev", "label":"Undevelopable", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsAA_lu.spatial.plot(map_widget = map_parcels_undevelopable,
                              renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                              unique_values=uv_lu,
                              arcade_expression=arcade_expression_lu,
                              default_symbol="" #don't include an 'other' category
                              )

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_undevelopable,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_undevelopable.layout.height='500px'
map_parcels_undevelopable.legend=True

#map title
#display(Markdown('<h2><center>Land Use - County ' + str(countyfilter) + '</center></h2>'))
display(Markdown('<h2><center>Existing Land Use</center></h2>'))

#display map
map_parcels_undevelopable

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


<h2><center>Existing Land Use</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Building Age

In [11]:
sdf_ParcelsAA.columns

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE'],
      dtype='object')

In [12]:
#set building age with base year 2019
sdf_ParcelsAA['BldAge'] = 0
sdf_ParcelsAA.loc[((sdf_ParcelsAA['yearbuilt'] > 0)), 'BldAge'] = 2019 - sdf_ParcelsAA['yearbuilt']
display(sdf_ParcelsAA)
display(sdf_ParcelsAA.columns)

,FID,Shape_Leng,Shape_Area,FID_Analys,AreaID,ClassID,FID_Parcel,parcel_id,parcel_id_,basebldg,...,sfvalue_re,sfvalue_co,value_res,value_com,ParcelAcre,PieceAcres,PiecePorti,Acres,SHAPE,BldAge
0,0,117.326952,835.775993,76,34,8,967,643515.0,643515.0,1.0,...,59.792523,0.0,137337.060741,0.0,0.206624,0.206525,0.999519,0.206525,"{""rings"": [[[413607.8234000001, 4462885.6642],...",17.0
1,1,109.679451,742.596041,76,34,8,968,643225.0,643225.0,1.0,...,53.048253,0.0,142099.315694,0.0,0.183590,0.183499,0.999508,0.183499,"{""rings"": [[[413635.0312000001, 4462961.596100...",20.0
2,2,110.755304,674.576194,76,34,8,969,697528.0,697528.0,1.0,...,66.142141,0.0,128523.105344,0.0,0.175349,0.166696,0.950653,0.166696,"{""rings"": [[[414230.7578999996, 4463098.232799...",14.0
3,3,72.509272,144.724562,76,34,8,970,697559.0,697559.0,0.0,...,0.000000,0.0,0.000000,0.0,0.285292,0.035762,0.125353,0.035762,"{""rings"": [[[414135.5011, 4462989.6206], [4141...",0.0
4,4,41.838689,100.414511,76,34,8,973,722478.0,722478.0,0.0,...,0.000000,0.0,0.000000,0.0,0.024905,0.024813,0.996292,0.024813,"{""rings"": [[[414093.95249999966, 4462731.52800...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125856,125856,260.101546,4210.391067,109,74,8,765605,109643.0,806702.0,0.0,...,0.000000,0.0,0.000000,0.0,1.905318,1.040410,0.546056,1.040410,"{""rings"": [[[432708.2054000003, 4505834.8017],...",0.0
125857,125857,431.493683,10458.588915,109,74,8,765627,109643.0,806724.0,0.0,...,0.000000,0.0,0.000000,0.0,2.586223,2.584374,0.999285,2.584374,"{""rings"": [[[432647.0683000004, 4505977.3521],...",0.0
125858,125858,410.754769,9101.330871,109,74,8,765628,109643.0,806725.0,0.0,...,0.000000,0.0,0.000000,0.0,4.293183,2.248988,0.523851,2.248988,"{""rings"": [[[432712.28000000026, 4505947.76], ...",0.0
125859,125859,395.479832,9369.595053,109,74,8,765650,109643.0,806747.0,0.0,...,0.000000,0.0,0.000000,0.0,2.809431,2.315277,0.824109,2.315277,"{""rings"": [[[432647.65189999994, 4506078.832],...",0.0


Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE', 'BldAge'],
      dtype='object')

In [25]:
#create map centered on Salt Lake
map_parcels_bldgage = gis.map('Salt Lake')


countyfilter = 2

#Filter by County
sdf_ParcelsAA_bldg = sdf_ParcelsAA[(sdf_ParcelsAA['county_id']==countyfilter) & (sdf_ParcelsAA['luTypeEx']!='Undevelopable')]
#sdf_ParcelsAA_lu

#expression to classify enrollment
arcade_expression_bldgage = ("var v = $feature.BldAge;"
                             "if      (v==0)  { return 'class1'; }"
                             "else if (v<40)  { return 'class2'; }"
                             "else if (v<80)  { return 'class3'; }"
                             "else if (v<150) { return 'class3'; }"
                             "else            { return 'class4'; }")

#symbology for enrollment classes
uv_public = [{"value":"class1", "label":"No Building"  , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class2", "label":"Less than 40" , "symbol":{"type":"esriSFS","color":[143,111,146,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class3", "label":"40 to 80"     , "symbol":{"type":"esriSFS","color":[155, 58, 75,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class4", "label":"80 to 150"    , "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
             {"value":"class4", "label":"More than 150", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsAA_bldg.spatial.plot(map_widget = map_parcels_bldgage,
                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                unique_values=uv_public,
                                arcade_expression=arcade_expression_bldgage,
                                default_symbol="" #don't include an 'other' category
                                )

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_bldgage,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_bldgage.layout.height='500px'
map_parcels_bldgage.legend=True

#map title
#display(Markdown('<h2><center>Building Age by Parcel - County ' + str(countyfilter) + '</center></h2>'))
display(Markdown('<h2><center>Building Age (Years) by Parcel</center></h2>'))

#display map
map_parcels_bldgage

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


<h2><center>Building Age (Years) by Parcel</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Parcel Value

In [14]:
sdf_ParcelsAA.columns

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE', 'BldAge'],
      dtype='object')

In [15]:
#set building age with base year 2019
sdf_ParcelsAA['ValuePerAcre'] = 0
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Residential')), 'ValuePerAcre'] = sdf_ParcelsAA['value_res'] / sdf_ParcelsAA['Acres']
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Commercial' )), 'ValuePerAcre'] = sdf_ParcelsAA['value_com'] / sdf_ParcelsAA['Acres']

#classify existing acreage by existing land use

sdf_ParcelsAA['acres_res'] = 0.0
sdf_ParcelsAA['acres_com'] = 0.0

sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Residential')), 'acres_res'] = sdf_ParcelsAA['Acres']
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Commercial' )), 'acres_com'] = sdf_ParcelsAA['Acres']

display(sdf_ParcelsAA)

,FID,Shape_Leng,Shape_Area,FID_Analys,AreaID,ClassID,FID_Parcel,parcel_id,parcel_id_,basebldg,...,value_com,ParcelAcre,PieceAcres,PiecePorti,Acres,SHAPE,BldAge,ValuePerAcre,acres_res,acres_com
0,0,117.326952,835.775993,76,34,8,967,643515.0,643515.0,1.0,...,0.0,0.206624,0.206525,0.999519,0.206525,"{""rings"": [[[413607.8234000001, 4462885.6642],...",17.0,664990.823797,0.206525,0.0
1,1,109.679451,742.596041,76,34,8,968,643225.0,643225.0,1.0,...,0.0,0.183590,0.183499,0.999508,0.183499,"{""rings"": [[[413635.0312000001, 4462961.596100...",20.0,774385.394871,0.183499,0.0
2,2,110.755304,674.576194,76,34,8,969,697528.0,697528.0,1.0,...,0.0,0.175349,0.166696,0.950653,0.166696,"{""rings"": [[[414230.7578999996, 4463098.232799...",14.0,771001.025112,0.166696,0.0
3,3,72.509272,144.724562,76,34,8,970,697559.0,697559.0,0.0,...,0.0,0.285292,0.035762,0.125353,0.035762,"{""rings"": [[[414135.5011, 4462989.6206], [4141...",0.0,0.000000,0.000000,0.0
4,4,41.838689,100.414511,76,34,8,973,722478.0,722478.0,0.0,...,0.0,0.024905,0.024813,0.996292,0.024813,"{""rings"": [[[414093.95249999966, 4462731.52800...",0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125856,125856,260.101546,4210.391067,109,74,8,765605,109643.0,806702.0,0.0,...,0.0,1.905318,1.040410,0.546056,1.040410,"{""rings"": [[[432708.2054000003, 4505834.8017],...",0.0,0.000000,0.000000,0.0
125857,125857,431.493683,10458.588915,109,74,8,765627,109643.0,806724.0,0.0,...,0.0,2.586223,2.584374,0.999285,2.584374,"{""rings"": [[[432647.0683000004, 4505977.3521],...",0.0,0.000000,0.000000,0.0
125858,125858,410.754769,9101.330871,109,74,8,765628,109643.0,806725.0,0.0,...,0.0,4.293183,2.248988,0.523851,2.248988,"{""rings"": [[[432712.28000000026, 4505947.76], ...",0.0,0.000000,0.000000,0.0
125859,125859,395.479832,9369.595053,109,74,8,765650,109643.0,806747.0,0.0,...,0.0,2.809431,2.315277,0.824109,2.315277,"{""rings"": [[[432647.65189999994, 4506078.832],...",0.0,0.000000,0.000000,0.0


In [16]:
#create map centered on Salt Lake
map_parcels_areas_value = gis.map('Salt Lake')



countyfilter = 2

#Filter by County
sdf_ParcelsAA_value = sdf_ParcelsAA[(sdf_ParcelsAA['county_id']==countyfilter) & (sdf_ParcelsAA['luTypeEx']!='Undevelopable')]
#sdf_ParcelsAA_lu

#expression to classify employment growth
#don't use value if 0
arcade_expression_value = ("var v = $feature.ValuePerAcre;"
                           "if      (v>0      && v<=100000) { return 'class4'; }"
                           "else if (v>100000 && v<=300000) { return 'class3'; }"
                           "else if (v>300000 && v<=500000) { return 'class2'; }"
                           "else if (v>500000             ) { return 'class1'; }")

#symbology for enrollment classes
uv_value = [{"value":"class1", "label":"More than 500k/acre" , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class2", "label":"$300k to $500k/acre" , "symbol":{"type":"esriSFS","color":[143,111,146,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class3", "label":"$100k to $300k/acre" , "symbol":{"type":"esriSFS","color":[155, 58, 75,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
            {"value":"class4", "label":"Less than $100k/acre", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsAA_value.spatial.plot(map_widget = map_parcels_areas_value,
                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                unique_values=uv_value,
                                arcade_expression=arcade_expression_value,
                                default_symbol="" #don't include an 'other' category
                                )

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_areas_value,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_areas_value.layout.height='500px'
map_parcels_areas_value.legend=True

#map title
display(Markdown('<h2><center>Value per Acre</center></h2>'))

#display map
map_parcels_areas_value

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


<h2><center>Value per Acre</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Development Categories

In [26]:
df_ClassParam = pd.read_csv(ClassParameters)
display(df_ClassParam)

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,ClassOrder
0,1,CRT|Metropolitan Center,0.40,0.60,350,8,1000,500,0.05,650000,400000,AND,60,150,30,150,1
1,5,LRT|Metropolitan Center,0.40,0.60,350,8,1000,500,0.05,650000,400000,AND,60,150,30,150,2
2,9,NONTOD|Metropolitan Center,0.45,0.55,170,4,1000,500,0.05,650000,400000,AND,60,150,30,150,3
3,3,CRT|Urban Center,0.50,0.50,130,3,1000,500,0.10,500000,300000,AND,70,300,30,300,4
4,7,LRT|Urban Center,0.50,0.50,130,3,1000,500,0.10,500000,300000,AND,70,300,30,300,5
5,10,NONTOD|Urban Center,0.55,0.45,70,2,1200,500,0.10,500000,300000,AND,70,300,30,300,6
6,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7
7,4,LRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,8
8,8,NONTOD|City Center,0.65,0.35,35,1,1200,600,0.10,500000,300000,AND,80,300,40,300,9
9,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10


In [27]:
#ReDevelop, NewDevelop, NoDevelop

sdf_ParcelsAAwParam = pd.DataFrame.merge(sdf_ParcelsAA,df_ClassParam,on='ClassID',how='left')
#display(sdf_ParcelsAAwParam)

#undeveloped
sdf_ParcelsDevCat = sdf_ParcelsAAwParam.copy()
sdf_ParcelsDevCat['DevCategory'] = 'Remain'
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['luTypeEx'] == 'Developable'), 'DevCategory'] = "Developable"

#check if redevelopable - step 1: building age
sdf_ParcelsDevCat['DevCategoryAge'] = ''
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['BldAge'] >= sdf_ParcelsDevCat['RedevBldgAgeRes_Low']) & (sdf_ParcelsDevCat['BldAge'] <= sdf_ParcelsDevCat['RedevBldgAgeRes_High']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Residential'), 'DevCategoryAge'] = "Redevelopable"
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['BldAge'] >= sdf_ParcelsDevCat['RedevBldgAgeCom_Low']) & (sdf_ParcelsDevCat['BldAge'] <= sdf_ParcelsDevCat['RedevBldgAgeCom_High']) & (sdf_ParcelsDevCat['luTypeEx'] != 'Residential'), 'DevCategoryAge'] = "Redevelopable"

#check if redevelopable - step 2: value
sdf_ParcelsDevCat['DevCategoryValue'] = ''
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['ValuePerAcre'] > 0) & (sdf_ParcelsDevCat['ValuePerAcre'] <= sdf_ParcelsDevCat['RedevValuePerAcreRes']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Residential'), 'DevCategoryValue'] = "Redevelopable"
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['ValuePerAcre'] > 0) & (sdf_ParcelsDevCat['ValuePerAcre'] <= sdf_ParcelsDevCat['RedevValuePerAcreCom']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Commercial' ), 'DevCategoryValue'] = "Redevelopable"

sdf_ParcelsDevCat.loc[ (sdf_ParcelsDevCat['RedevAndOr'] =='AND') &  (sdf_ParcelsDevCat['DevCategoryAge'] == 'Redevelopable') & (sdf_ParcelsDevCat['DevCategoryValue'] == 'Redevelopable')  , 'DevCategory'] = "Redevelopable"
sdf_ParcelsDevCat.loc[((sdf_ParcelsDevCat['RedevAndOr'] =='OR' ) & ((sdf_ParcelsDevCat['DevCategoryAge'] == 'Redevelopable') | (sdf_ParcelsDevCat['DevCategoryValue'] == 'Redevelopable'))), 'DevCategory'] = "Redevelopable"

#undevelopable acres
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['luTypeEx'] == 'Undevelopable'), 'DevCategory'] = "Undevelopable"

display(sdf_ParcelsDevCat[['yearbuilt','luTypeEx','ValuePerAcre','BldAge','DevCategory']])
sdf_ParcelsDevCat.columns

,yearbuilt,luTypeEx,ValuePerAcre,BldAge,DevCategory
0,2002.0,Residential,664990.823797,17.0,Remain
1,1999.0,Residential,774385.394871,20.0,Remain
2,2005.0,Residential,771001.025112,14.0,Remain
3,0.0,Developable,0.000000,0.0,Developable
4,0.0,Developable,0.000000,0.0,Developable
...,...,...,...,...,...
125856,0.0,Developable,0.000000,0.0,Developable
125857,0.0,Developable,0.000000,0.0,Developable
125858,0.0,Developable,0.000000,0.0,Developable
125859,0.0,Developable,0.000000,0.0,Developable


Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE', 'BldAge', 'ValuePerAcre', 'acres_res',
       'acres_com', 'ClassDescription', 'SFSplitRes', 'SFSplitCom',
       'CapacityRes_DUA', 'CapacityCom_FAR', 'SFperHH', 'SFperEmp',
       'PercentOpenSpace', 'RedevValuePerAcreRes', 'RedevValuePerAcreCom',
       'RedevAndOr', 'RedevBldgAgeRes_Low', 'RedevBldgAgeRes_High',
       'RedevBldgAgeCom_Low', 'RedevBldgAgeCom_High', 'ClassOrder',
       'DevCategory', 'DevCategoryAge', 'DevCategoryValue'],
      dtype='object')

In [28]:
display(df_ClassParam)
df_ClassParam.dtypes

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,ClassOrder
0,1,CRT|Metropolitan Center,0.40,0.60,350,8,1000,500,0.05,650000,400000,AND,60,150,30,150,1
1,5,LRT|Metropolitan Center,0.40,0.60,350,8,1000,500,0.05,650000,400000,AND,60,150,30,150,2
2,9,NONTOD|Metropolitan Center,0.45,0.55,170,4,1000,500,0.05,650000,400000,AND,60,150,30,150,3
3,3,CRT|Urban Center,0.50,0.50,130,3,1000,500,0.10,500000,300000,AND,70,300,30,300,4
4,7,LRT|Urban Center,0.50,0.50,130,3,1000,500,0.10,500000,300000,AND,70,300,30,300,5
5,10,NONTOD|Urban Center,0.55,0.45,70,2,1200,500,0.10,500000,300000,AND,70,300,30,300,6
6,0,CRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,7
7,4,LRT|City Center,0.60,0.40,70,2,1200,600,0.10,500000,300000,AND,70,300,30,300,8
8,8,NONTOD|City Center,0.65,0.35,35,1,1200,600,0.10,500000,300000,AND,80,300,40,300,9
9,2,CRT|NA,0.70,0.30,35,1,1200,600,0.15,500000,300000,AND,70,300,30,300,10


ClassID                   int64
ClassDescription         object
SFSplitRes              float64
SFSplitCom              float64
CapacityRes_DUA           int64
CapacityCom_FAR           int64
SFperHH                   int64
SFperEmp                  int64
PercentOpenSpace        float64
RedevValuePerAcreRes      int64
RedevValuePerAcreCom      int64
RedevAndOr               object
RedevBldgAgeRes_Low       int64
RedevBldgAgeRes_High      int64
RedevBldgAgeCom_Low       int64
RedevBldgAgeCom_High      int64
ClassOrder                int64
dtype: object

In [29]:
countyfilter = 2

#Filter by County
sdf_ParcelsDevCat_filter = sdf_ParcelsDevCat.copy()
sdf_ParcelsDevCat_filter = sdf_ParcelsDevCat_filter[sdf_ParcelsDevCat_filter['county_id']==countyfilter]

#create map centered on Salt Lake
map_parcels_dev = gis.map('Salt Lake')

#expression to classify enrollment
arcade_expression_dev = ("var v = $feature.DevCategory;"
                             "if      (v=='Undevelopable') { return 'class2'; }"
                             "else if (v=='Remain'       ) { return 'class2'; }"
                             "else if (v=='Redevelopable') { return 'class3'; }"
                             "else if (v=='Developable'  ) { return 'class4'; }"
                        )

#symbology for enrollment classes
uv_dev = [{"value":"class4", "label":"Developable"  , "symbol":{"type":"esriSFS","color":[  0,  0,255,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"class3", "label":"Redevelopable", "symbol":{"type":"esriSFS","color":[128,  0,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"class2", "label":"To Remain"    , "symbol":{"type":"esriSFS","color":[180,180,180,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#define sdf layer
sdf_ParcelsDevCat_filter.spatial.plot(map_widget = map_parcels_dev,
                                      renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                      unique_values=uv_dev,
                                      arcade_expression=arcade_expression_dev,
                                      default_symbol="" #don't include an 'other' category
                                      )
#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]


#define sdf layer
sdf_AA.spatial.plot(map_widget = map_parcels_dev,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_parcels_dev.layout.height='500px'
map_parcels_dev.legend=True

#map title
display(Markdown('<h2><center>Development Category - County ' + str(countyfilter) + '</center></h2>'))

#display map
map_parcels_dev

<h2><center>Development Category - County 2</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

In [31]:
#aggregate parcels into analysis areas / classes
df_AADevCat = sdf_ParcelsDevCat.groupby(['AreaID','ClassID','county_id','DevCategory'],as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "Acres":[np.sum], "sf_res":[np.sum], "sf_com":[np.sum], "acres_res":[np.sum], "acres_com":[np.sum]})
df_AADevCat.columns = df_AADevCat.columns.droplevel(1)
display(df_AADevCat)


df_AADevCat.to_csv(os.path.join(dir_results, r'AnalysisAreas_byDevCategory.csv'),index=False)


,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com
0,0,8,1.0,Developable,48.0,0.000000,0.000000,14.636327,0.000000,0.000000,0.000000,0.000000
1,0,8,1.0,Redevelopable,13.0,108.943539,0.249295,15.238659,498.590239,46916.015226,0.070587,15.168072
2,0,8,1.0,Remain,56.0,1257.657888,8.624897,40.616695,10809.745772,586355.164767,1.581190,39.035505
3,0,8,1.0,Undevelopable,17.0,0.000000,27.973801,8.426638,42936.427209,0.000000,0.000000,0.000000
4,1,6,2.0,Developable,23.0,0.000000,0.000000,26.560252,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
690,157,8,2.0,Undevelopable,58.0,492.928852,11.790533,123.509517,14980.641308,174191.014758,0.000000,0.000000
691,158,2,1.0,Developable,94.0,0.000000,0.000000,45.785647,0.000000,0.000000,0.000000,0.000000
692,158,2,1.0,Redevelopable,15.0,75.191049,3.996853,10.294118,6684.737243,47550.563170,1.388371,8.905747
693,158,2,1.0,Remain,436.0,714.783931,392.240448,112.392493,491691.829693,393588.574212,79.871712,32.520780
